## Задание 2. Распилы льда

#### Описание задачи:

Большинство метеонаблюдений до сих пор сначала заносится в тетрадку, а потом передаётся по рации. В результате после оцифровки в датасете о распилах льда на реке Лена оказываются ошибки. Где они и как с ними бороться?

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim

In [2]:
# Чтение данных

# Список листов excel
sheets = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

# Заготовка датафрейма
df = pd.DataFrame()

# Постраничное чтение excel в цикле, добавление отметки о годе и добавление в итоговый датафрейм
for sheet in sheets:
    new_sheet = pd.read_excel('Мероприятия для МЧС (1803 - Лена).xlsx', sheet_name=sheet, header=[0,1])
    new_sheet['Год'] = sheet
    df = df.append(new_sheet)
    
df.head()

Расположение                Тип  \
  Unnamed: 0_level_1 Unnamed: 1_level_1   
0              Ленск               СУБВ   
1              Ленск               СУБВ   
2             Якутск               СУБВ   
3           Покровск               СУБВ   
4          Олёкминск                ГТС   

                           Наименование мероприятия        Координаты  \
                                  Unnamed: 2_level_1 Широта, Градусов   
0  Ослабление прочности льда на р.Нюя в Ленском р...               61   
1  Определение границ водоохранной зоны и прибреж...               35   
2  Определение границ водоохранной зоны и прибреж...               62   
3  Определение границ водоохранной зоны и прибреж...                0   
4  Ослабление прочности льда на р.Лена в Олекминс...               60   

                                                                 \
  Широта, Минут Широта, Секунд Долгота, Градусов Долгота, Минут   
0            22             42               114             33   
1            43             60               114             56   
2             1             38               129             43   
3            29             61               129              9   
4            23              0               120             26   

                    Влияние на водный объект  \
  Долгота, Секунд         Unnamed: 9_level_1   
0              54  Ослабление прочности льда   
1              43                Нет влияния   
2              55                Нет влияния   
3               3                Нет влияния   
4               0  Ослабление прочности льда   

                                                         Год  
  СУБВ - мероприятие осуществляется за счет субвенций         
0                        ГТС - основная деятельность    2011  
1                                                NaN    2011  
2                                                NaN    2011  
3                                                NaN    2011  
4                                                NaN    2011

In [3]:
# Приведение названий столбцов к единому формату
df.rename(columns={'Unnamed: 0_level_1': 'Расположение', 'Unnamed: 1_level_1': 'Тип', 'Unnamed: 2_level_1':
                  'Наименование мероприятия', 'Unnamed: 9_level_1': 'Влияние на водный объект', '': 'Год'},
         inplace=True)
df.columns = df.columns.droplevel(0)

df.head()

,Расположение,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,СУБВ - мероприятие осуществляется за счет субвенций,Год
0,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61,22,42,114,33,54,Ослабление прочности льда,ГТС - основная деятельность,2011
1,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,35,43,60,114,56,43,Нет влияния,NaN,2011
2,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,62,1,38,129,43,55,Нет влияния,NaN,2011
3,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0,29,61,129,9,3,Нет влияния,NaN,2011
4,Олёкминск,ГТС,Ослабление прочности льда на р.Лена в Олекминс...,60,23,0,120,26,0,Ослабление прочности льда,NaN,2011


In [4]:
# Общая информация по датафрейму: размеры, типы данных, пропущенные значения
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110 entries, 0 to 7
Data columns (total 12 columns):
 #   Column                                                Non-Null Count  Dtype 
---  ------                                                --------------  ----- 
 0   Расположение                                          110 non-null    object
 1   Тип                                                   110 non-null    object
 2   Наименование мероприятия                              110 non-null    object
 3   Широта, Градусов                                      110 non-null    object
 4   Широта, Минут                                         110 non-null    object
 5   Широта, Секунд                                        110 non-null    object
 6   Долгота, Градусов                                     110 non-null    object
 7   Долгота, Минут                                        110 non-null    object
 8   Долгота, Секунд                                       110 non-null    ob

### Промежуточные выводы

1. Все данные имеют тип object. Для удобства нужно будет преобразовать часть из них в числовые типы

2. Предпоследний столбец можно удалить из датафрейма. Если обратиться к исходному excel-файлу, можно увидеть, что данный столбец является просто расшифровкой значений для столбца "тип".

3. В столбце "Влияние на водный объект" есть пропущенные значения. 

4. В датафрейме много текстовых столбцов. При ручном вводе там могли оказаться опечатки. Это необходимо проверить

5. (дополнительно опционально) Столбцы названы по-русски. Это может быть неудобно для аналитики в python, так как постоянно придется переключать раскладку клавиатуры. Для удобства можно было бы переименовать столбцы на английские названия 

In [5]:
# удаление лишнего столбца
df.drop(df.columns[-2], axis=1, inplace=True)
df.head()

,Расположение,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год
0,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61,22,42,114,33,54,Ослабление прочности льда,2011
1,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,35,43,60,114,56,43,Нет влияния,2011
2,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,62,1,38,129,43,55,Нет влияния,2011
3,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0,29,61,129,9,3,Нет влияния,2011
4,Олёкминск,ГТС,Ослабление прочности льда на р.Лена в Олекминс...,60,23,0,120,26,0,Ослабление прочности льда,2011


### Шаг 1. Работа с ошибками числовых данных

In [6]:
# Проверим, все ли значения координат числовые: выведем все уникальные значения по столбцу "широта"
df['Широта, Градусов'].unique()

array([61, 35, 62, 0, 60, 1, 54, 59, 36, 2, 38, 7, 33, 42, 63, 13, 'НД',
       57, 29, 27, 15, 71], dtype=object)

In [7]:
# по столбцу "долгота"
df['Долгота, Градусов'].unique()

array([114, 129, 120, 115, 127, 109, 'НД', 108, 116, 122, 130, 131, 152,
       156, 119, 126, 133, 0], dtype=object)

In [8]:
# Помимо числовых, есть отметка "НД" (по-видимому, "нет данных"), при преобразовании необходимо это учитывать

# Также есть нулевое значение широты, чего быть не может, так как нулевой градус широты соответствует экватору. 
# Здесь явно ошибка

# Нулевое значение меридиана (долгота) также должно быть ошибкой, так как нулевой меридиан не проходит по 
# исследуемым территориям

# Преобразуем столбцы с координатами в числовой тип. Отсутствующие значения заменяем на NaN
df.iloc[:, 3:9] = df.iloc[:, 3:9].apply(pd.to_numeric, errors='coerce')
df.head()

,Расположение,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год
0,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2011
1,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,35.0,43.0,60.0,114.0,56.0,43.0,Нет влияния,2011
2,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,62.0,1.0,38.0,129.0,43.0,55.0,Нет влияния,2011
3,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0.0,29.0,61.0,129.0,9.0,3.0,Нет влияния,2011
4,Олёкминск,ГТС,Ослабление прочности льда на р.Лена в Олекминс...,60.0,23.0,0.0,120.0,26.0,0.0,Ослабление прочности льда,2011


In [9]:
# Выведем строки с отсутствующими значениями координат, а также с нулевыми значениями широты и долготы
df[(df['Широта, Градусов'].isnull()) | (
    df['Долгота, Градусов'].isnull()) | (
    df['Широта, Градусов'] == 0) | (
    df['Долгота, Градусов'] == 0)]

,Расположение,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год
3,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0.0,29.0,61.0,129.0,9.0,3.0,Нет влияния,2011
6,Саха (Якутия),СУБВ,Разработка проекта СКИОВО бассейна реки Лена,0.0,2.0,62.0,129.0,44.0,0.0,Нет влияния,2011
6,Покровск,СУБВ,Закрепление на местности границ водоохранной ...,0.0,29.0,61.0,129.0,9.0,3.0,Нет влияния,2012
11,Саха (Якутия),СУБВ,Разработка проекта СКИОВО бассейна реки Лена.,0.0,2.0,62.0,129.0,44.0,0.0,Нет влияния,2012
14,Саха (Якутия),СУБВ,Исследование и оценка воздействия мероприятий ...,0.0,2.0,62.0,129.0,44.0,0.0,Нет влияния,2012
1,Саха (Якутия),СУБВ,"Разработка проекта ""Расчистка и дноуглубление ...",NaN,NaN,NaN,NaN,NaN,NaN,Нет влияния,2014
11,Олёкминск,СУБВ,Расчистка и дноуглубление русла р. Алалайка в ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
6,Саха (Якутия),СУБВ,Изучение гидроледотермических условий формиров...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2017


### Основные выводы по ошибкам числовых данных:

* В датафрейме присутствуют ошибки ввода координат. Наиболее явная ошибка - отсутствие значений. В исходном датафрейме они указаны как "НД". Также ошибкой является значение 0 в градусах широты и долготы, так как на исследуемых территориях не может быть таких значений широты и долготы (нулевая широта - экватор, нулевой меридиан - Гринвич, Великобритания). 


* Также может возникнуть проблема, связанная с единообразием предоставления информации о координатах. Например, в датафрейме есть строки с нулевым значением *секунды* широты и долготы. Это может свидетельствовать не только о том, что на данной территории действительно такая координата, но также и о том, что некоторые территории предоставляют неполную информацию о координатах. Здесь необходимо как-то стандартизировать информацию: либо вообще не уточнять координаты до секунд, либо восстанавливать точную координату по всем территориям


* При этом перечисленное - это только наиболее явные ошибки в координатах. Также могут присутствовать другие ошибки ввода координат (например, широта 1, 2 - это так же ошибки)


* Также координатами в том виде, в котором они представлены, то есть в шести столбцах, неудобно пользоваться. Было бы удобнее работать с полной координатой в одной ячейке.


* Выходом может быть использование специальных библиотек и модулей для работы с геолокацией (например, geopy). Можно создать столбец, куда по названию города загрузить точную координату города. Однако мероприятия не всегда проводятся в черте города, поэтому точность до минут и секунд так восстановить не удастстся. Но можно восстановить хотя бы корректные координаты широты и долготы. (Более точно геолокацию может помочь определить столбец "Наименование мероприятий" - об этом ниже)

In [10]:
# Пример нахождения координаты по названию города
geolocator = Nominatim()
location = geolocator.geocode('Якутск')
location.latitude, location.longitude

(62.027287, 129.732086)

### Шаг 2. Работа с ошибками текстовых данных

In [11]:
# В текстовые столбцы могли попасть опечатки. Проверим все уникальные значения в таких столбцах
df['Расположение'].unique()

array(['Ленск', 'Якутск', 'Покровск', 'Олёкминск', 'Саха (Якутия)',
       'Якутстк', 'Олекминск', 'Сангар', 'г. Киренск', 'Саха Якутия'],
      dtype=object)

In [12]:
# избавимся от опечаток и приведем города к единообразию

# Саха (Якутия) тоже приведем к единому виду, однако надо понимать, что здесь все равно остается ошибка, 
# так как это не город, а субъект. Потенциально можно восстановить конкретный населенный пункт по описанию 
# мероприятия

# словарь для замены
errors_dict = {'Якутстк': 'Якутск'
               , 'Олёкминск': 'Олекминск'
               , 'г. Киренск': 'Киренск'
               , 'Саха Якутия': 'Саха (Якутия)'}
# замена
df = df.replace({'Расположение': errors_dict})

# проверка результата
df['Расположение'].unique()

array(['Ленск', 'Якутск', 'Покровск', 'Олекминск', 'Саха (Якутия)',
       'Сангар', 'Киренск'], dtype=object)

In [13]:
df['Тип'].unique()

array(['СУБВ', 'ГТС'], dtype=object)

In [14]:
df['Влияние на водный объект'].unique()

array(['Ослабление прочности льда', 'Нет влияния', nan], dtype=object)

In [21]:
# Посмотрим, какие названия мероприятий соотвутствуют влиянию "ослабление прочности льда" в столбце "Влияние
# на водный объект"
df['Наименование мероприятия'][df['Влияние на водный объект'] == 'Ослабление прочности льда'].unique()

array(['Ослабление прочности льда на р.Нюя в Ленском районе, Республика Саха (Якутия)',
       'Ослабление прочности льда на р.Лена в Олекминском улусе, Республика Саха (Якутия)',
       'Ослабление прочности льда на р.Лена в Ленском улусе, Республика Саха (Якутия)',
       'Ослабление прочности льда на р.Лена в районе г.Якутск, Республика Саха (Якутия)',
       'Ослабление прочности льда на р. Нюя в Ленском улусе, Республика Саха (Якутия)\n',
       'Ослабление прочности льда на р.Лена в Хангаласском улусе, Республика Саха (Якутия)',
       'Ослабление прочности льда на р.Лена в Кобяйском улусе, Республика Саха (Якутия)',
       'Ослабление прочности льда на р. Нюя в Ленском улусе, Республики Саха (Якутия)',
       '#Ослабление прочности льда на р. Лена в Ленском улусе, Республика Саха (Якутия)',
       '#Ослабление прочности льда на р. Лена в Олекминском улусе, Республика Саха (Якутия)',
       '#Ослабление прочности льда на р. Лена в Хангаласском улусе, Республика Саха (Якутия)',
  

In [22]:
# количество уникальных значений наименований мероприятий
len(df['Наименование мероприятия'].unique())

77

### Основные выводы по текстовым столбцам:

* В датасете действительно есть опечатки в столбце с названиями населенных пунктов. Помимо опечаток, отсутствует единообразие ввода (Саха(Якутия) и Саха Якутия или Олекминск и Олёкминск - это одно и то же, "г." для города указан только в одном случае). Также Саха - это название региона, а всё остальное в датасете - это города. Эти ошибки необходимо исправлять и приводить названия к единому виду.


* Столбцы "тип" и "влияние на водный объект" являются категориальными, где возможны всего по два варианта: СУБВ, ГТС в типе и "Ослабление прочности льда", "Нет влияния" во влиянии. При необходимости, для дальнейшего анализа можно закодировать данную информацию (например, числовыми характеристиками 0 - нет влияния, 1 - ослабление льда)


* Со столбцом "Наименование мероприятия" работать сложнее, так как по сути он является полным описанием произведенных действий. Как следствие, здесь много уникальных значений (потому что даже отличие в один знак препинания делает два совершенно одинаковых текста разными - такие примеры в датасете есть), а значит, датафрейм будет сложно группировать по проведенным мероприятиям. Необходимо унифицировать данный столбец, например, выделив ключевые слова в каждом мероприятии, убрав все знаки препинания, высокий регистр, предлоги, союзы и пр. Для этого можно использовать регулярные выражения и другие инструменты для работы со строками. При этом сам исходный столбец "Наименования мероприятия" также стоит оставить, так как к нему можно будет обращаться при необходимости полного описания мероприятия.


* Также в описаниях мероприятий указаны конкретные населенные пункты. Это позволит восстановить населенные пункты в случаях, когда указан только регион Саха (Якутия). Как следствие, можно восстановить и точные координаты при помощи библиотеки geopy


* Описания мероприятий также могут помочь восстановить пропущенные значения в столбце "влияние на водный объект". Так, во всех строках с "ослаблением прочности льда" соответствующее мероприятие называется так же. Значит, при отсутствии в описании мероприятия слов "ослабление прочности льда" (что можно найти при помощи регулярных выражений, например) в пропущенные значения столбца "Влияние на водный объект" можно поставить "нет влияния"